In [6]:
import os
import glob
import keras

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from skimage import transform
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [7]:
num_final_hidden_layer = 32
num_classes = 2
batch_size = 32

checkpoint_dir = '../experiments/baseline'
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [8]:
model = Sequential()
model.add(Conv2D(
        32, kernel_size=4, strides=2, activation='relu',
        input_shape=(128, 128, 3), padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(512, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(num_final_hidden_layer, kernel_size=4))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [9]:
stopping = EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_dir, 'checkpoint'),
    monitor='val_acc', save_best_only=True)
model.compile(optimizer=Adam(lr=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
datagen = ImageDataGenerator(rescale=1./255)
train_datagen = datagen.flow_from_directory(
        '/home/supervised-vae/data/processed/train',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

val_datagen = datagen.flow_from_directory(
        '/home/supervised-vae/data/processed/validation',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

test_datagen = datagen.flow_from_directory(
        '/home/supervised-vae/data/processed/test',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

Found 460749 images belonging to 2 classes.
Found 25597 images belonging to 2 classes.
Found 25600 images belonging to 2 classes.


In [15]:
history = model.fit_generator(
    train_datagen, validation_data=val_datagen,
    callbacks=[stopping, checkpoint], epochs=100,
    steps_per_epoch=len(train_datagen),
    validation_steps=len(val_datagen))

Epoch 1/100
14399/14399 [==============================] - 1571s 109ms/step - loss: 0.5409 - acc: 0.7393 - val_loss: 0.5540 - val_acc: 0.7367
Epoch 2/100
14399/14399 [==============================] - 1574s 109ms/step - loss: 0.5108 - acc: 0.7573 - val_loss: 0.5821 - val_acc: 0.7207
Epoch 3/100
14399/14399 [==============================] - 1575s 109ms/step - loss: 0.4856 - acc: 0.7712 - val_loss: 0.5720 - val_acc: 0.7274
Epoch 4/100
14399/14399 [==============================] - 1579s 110ms/step - loss: 0.4618 - acc: 0.7820 - val_loss: 0.5879 - val_acc: 0.7181


In [16]:
result = model.evaluate_generator(test_datagen, steps=len(test_datagen))
result

[0.5511836166679859, 0.7396484375]